In [1]:
import openeo

connection = openeo.connect(url='openeo.dataspace.copernicus.eu')

connection.authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=GSER-KGSL 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [5]:
import openeo, datetime
from openeo.processes import *

# Connect & authenticate (OIDC in browser)
conn = openeo.connect("openeo.cloud").authenticate_oidc()

# AOI & period
bbox   = {"west": -1.2, "east": -0.6, "south": 39.2, "north": 39.8}
start, end = "2024-09-01", "2024-10-01"

# 1) 3-hourly precipitation cube (MSWX)
pr = conn.load_collection(
        "MSWX_3H",
        spatial_extent=bbox,
        temporal_extent=[start, end],
        bands=["precipitation"]
)

# aggregate to daily totals
daily_pr = pr.aggregate_temporal_period("day", reducer=sum)

# 2) Binary flood extent from GFM (Sentinel-1 ensemble)
flood = conn.load_collection(
        "GFM",
        spatial_extent=bbox,
        temporal_extent=[start, end],
        bands=["ensemble_flood_extent"]
).reduce_dimension(dimension="t", reducer=any)

# 3) Stack and export
cube = daily_pr.add_dimension(dimension="bands", label="precip") \
               .merge_cubes(flood.rename_labels("bands", ["ensemble_flood_extent"], ["flood"]))

cube.save_result(format="GTiff", options={"tile_grid": "wgs84-1degree"})


Visit https://aai.egi.eu/device?user_code=YZAZ-YORS 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


OpenEoApiError: [404] CollectionNotFound: Collection 'MSWX_3H' does not exist. (ref: r-2505180736024b53926a410bc3bf9428)

In [7]:
import openeo, datetime
from openeo.processes import *

conn = openeo.connect("openeo.cloud").authenticate_oidc()

bbox = {"west": -1.2, "east": -0.6, "south": 39.2, "north": 39.8}
period = ["2024-09-01", "2024-10-01"]

# (1)  daily precip totals from AgERA5
pr = (conn.load_collection("AGERA5",
        spatial_extent=bbox,
        temporal_extent=period,
        bands=["precipitation"])
      .aggregate_temporal_period("day", reducer="sum"))

# (2)  NRT flood extent (Sentinel-1 ensemble)
flood = (conn.load_collection("GFM",
         spatial_extent=bbox,
         temporal_extent=period,
         bands=["ensemble_flood_extent"])
         .reduce_dimension("t", reducer="any"))

# (3)  merge & save
cube = pr.merge_cubes(flood.rename_labels("bands", ["ensemble_flood_extent"], ["flood"]))
cube.save_result(format="GTiff", options={"tile_grid": "wgs84-1degree"})


Authenticated using refresh token.


ValueError: Invalid band name/index 'precipitation'. Valid names: ['dewpoint-temperature', 'precipitation-flux', 'solar-radiation-flux', 'temperature-max', 'temperature-mean', 'temperature-min', 'vapour-pressure', 'wind-speed']